# Archivematica and Omeka S

> Fill in a module description here

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext dotenv
%dotenv

In [ ]:
#| export
import shutil
import glob
from pprint import pprint
import os
import bs4
import json
import pandas as pd
from pandas import json_normalize
from tqdm import tqdm
from copy import deepcopy

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
#| export

from omeka_s_tools2.api import OmekaAPIClient

class OmekaClient:
  term = "dcterms:identifier"

  def __init__(self, API_URL, KEY_IDENTITY, KEY_CREDENTIAL):
      self.API_URL = API_URL
      self.KEY_IDENTITY = KEY_IDENTITY
      self.KEY_CREDENTIAL = KEY_CREDENTIAL

  def reset(self):
      self.omeka_auth = OmekaAPIClient(
          api_url=self.API_URL,
          key_identity=self.KEY_IDENTITY, 
          key_credential=self.KEY_CREDENTIAL
      )

  def check(self):
      print(self.API_URL, self.KEY_IDENTITY, self.KEY_CREDENTIAL)

  def upload(self, metadata_path, media_path, is_public):
      '''アイテムとメディアの登録
      '''

      self.is_public = is_public

      # アイテムの登録
      self.upload_items(metadata_path)        

      # メディアの登録
      items = self.convertCsv2Json(media_path)
      self.upload_media(items, media_path)

  def upload_items(self, metadata_path):
      '''アイテムの登録
      '''
      print("アイテムの登録")
      items = self.convertCsv2Json(metadata_path)
      for item in tqdm(items):
          self.update(item)

  def convertCsv2Json(self, csv_path):
      df = pd.read_csv(csv_path)
      # print(df)

      items = []
      for index, row in df.iterrows():
          test_item = {}
          for column_name, item in df.iteritems():
              if column_name not in test_item:
                  test_item[column_name] = []
              value = row[column_name]
              if pd.isnull(value):
                  continue
              test_item[column_name].append(value)

          # pprint(test_item)
          items.append(test_item)
          # break

      # pprint(items)

      return items

  def getExistingsValues(self, term, value):
      url = f"{self.API_URL}/items?property[0][property]={term}&property[0][type]=eq&property[0][text]={value}&key_identity={self.KEY_IDENTITY}&key_credential={self.KEY_CREDENTIAL}"
      import requests
      df = requests.get(url).json()
      return df

  def update(self, test_item):

      term = self.term

      self.reset()
      omeka_auth = self.omeka_auth

      local_item = deepcopy(test_item)

      replaced_fields = {}

      for field in local_item:
          if "^^resource" in field:
              values = local_item[field]

              resources = []

              for value in values:
                  existing_values = self.getExistingsValues(term, value)

                  if len(existing_values) > 0:
                      resources.append({
                          "type": "resource",
                          "value": existing_values[0]["o:id"]
                      })

              '''
              del local_item[field]
              local_item[field.split(" ")[0]] = resources
              '''
              replaced_fields[field] = resources

      for field in replaced_fields:
          del local_item[field]
          local_item[field.split(" ")[0]] = replaced_fields[field]

      payload = omeka_auth.prepare_item_payload(local_item)
      
      # 既存のIDを取得
      id = local_item[term][0]

      # 既存のIDを持つアイテムの取得
      results = self.getExistingsValues(term, id)

      flg = True

      if flg:

          # 既に存在する場合は更新
          if len(results) > 0:
              payload_old = results[0]
              for key in payload:
                  payload_old[key] = payload[key]

              payload = payload_old

      payload["is_public"] = self.is_public

      # バグ修正

      for field in payload:
          values = payload[field]
          if type(values) is not list:
              continue
          for value in values:
              # @value を value_resource_id に置換する
              if "type" in value and value["type"] == "resource" and "@value" in value:
                  value["value_resource_id"] = value["@value"]

      if len(results) > 0:
          omeka_auth.update_resource(payload, 'items')
      else:
          omeka_auth.add_item(payload)

  def upload_media(self, items, media_path):
      dir = os.path.dirname(media_path)
      id_map = {}
      term = self.term

      print("\nIDの取得\n")

      for item in tqdm(items):
          self.reset()

          id = item["item"][0]

          if id in id_map:
              continue

          arr = self.getExistingsValues(term, id)

          if len(arr) > 0:
              id_map[id] = arr[0]["o:id"]

      print("\nメディアの登録\n")

      for item in tqdm(items):
          id = item["item"][0]

          if id not in id_map:
              continue

          media_path = f'{dir}/{item["path"][0]}'
          oid = id_map[id]

          self.reset()
          self.omeka_auth.add_media_to_item(oid, media_path)

class ArchiveMaticaOmeka:
  TMP_DIR_PATH = f"tmp"

  def __init__(self, dip_zip_file_path, mapping_json_file_path, task_id="test", debug=False):
    self.dip_zip_file_path = dip_zip_file_path
    mapping_json_file_path = mapping_json_file_path

    with open(mapping_json_file_path) as f:
      self.mapping = json.load(f)

    self.task_id = task_id

    tmp_dir_path = f"{self.TMP_DIR_PATH}/{task_id}"

    self.tmp_dir_path = tmp_dir_path

    if os.path.exists(tmp_dir_path):
      shutil.rmtree(tmp_dir_path)

    self.debug = debug

  def unpackArchive(self):
    """zipファイルの展開
    """

    shutil.unpack_archive(self.dip_zip_file_path, self.tmp_dir_path)

  ## 
  def getMetsFilePath(self):
    """metsファイルのパスの取得
    """

    files = glob.glob(f"{self.tmp_dir_path}/*/*.xml")
    # pprint(files)

    mets_path = None
    for file in files:
      if "METS" in file:
        mets_path = file

    # print(mets_path)
    self.mets_path = mets_path

    # スクレイピング対象のhtmlファイルからsoupを作成
    soup = bs4.BeautifulSoup(open(self.mets_path), 'xml')
    self.soup = soup

    if self.debug:
      print("-----\ngetMetsFilePath\n-----")
      print(soup.prettify())

  ## dmdSecの情報の取得
  def getDmdSec(self):
    """dmdSecの情報の取得
    """
    ## metsファイルのパスの取得

    mapping = self.mapping

    dmdSecs = self.soup.find_all("mets:dmdSec")

    items = {}
    # 順序を保持するためにlistを使用
    item_ids = []
    mappings = {}

    for dmdSec in dmdSecs:
      dmdSec_id = dmdSec.get("ID")
      mdWrap = dmdSec.find("mets:mdWrap")
      mdType = mdWrap.get("MDTYPE")
      # print(mdType)

      if mdType == "DC":
        item = {}
        # items.append(item)
        dc = mdWrap.find("dcterms:dublincore")
        metadata = dc.findChildren()
        # pprint(metadata)

        for m in metadata:
          name = "dc:" + m.name
          value = m.text
          # print(name, value)

          if value != "" and name in mapping:
            p_id = mapping[name]
            
            # print("*", p_id)

            if p_id == "dcterms:identifier":
              item_ids.append(value)
              id = value
            else:
              item[p_id] = value

        items[id] = item

        mappings[dmdSec_id] = id

      if mdType == "OTHER":
        # item = {}
        # items.append(item)
        dc = mdWrap.find("mets:xmlData")
        metadata = dc.findChildren()
        # pprint(metadata)

        for m in metadata:
          name = m.name
          value = m.text
          # print(name, value)

          if value != "" and name in mapping:
            p_id = mapping[name]
            
            item[p_id] = value

        items[id] = item

    # pprint(items)

    self.items = items
    self.item_ids = item_ids
    self.dmd_sec_mappings = mappings

    if self.debug:
      print("-----\nmappings\ndmdSecとdcterms:identifierの関係\n-----")
      pprint(mappings)

    if self.debug:
      print("-----\ngetDmdSec\ndcterms:identifier毎のメタデータ\n-----")
      pprint(items)

  ## 
  def getStructMap(self):
    """structMapの情報の取得
    """

    dmd_sec_mappings = self.dmd_sec_mappings

    fptrs = self.soup.find_all("mets:fptr")

    structs = []

    for fptr in fptrs:
      file_id = fptr.get("FILEID")

      div_item = fptr.parent

      # フラット
      if "DMDID" in div_item.attrs:
        dmd_ids = div_item.get("DMDID").split(" ")
        for dmd_id in dmd_ids:
          if dmd_id not in dmd_sec_mappings:
            continue

          item_id = dmd_sec_mappings[dmd_id]
      else:


        label_item = div_item.get("LABEL")

        div_item_parent = div_item.parent

        if label_item in ["metadata.csv", "METS.xml", "directory_tree.txt"]:
          continue

        item_id = div_item_parent.get("LABEL")

        if item_id in ["OCRfiles"]:
          continue

      structs.append({
        "file": file_id,
        "item_id": item_id  # label_item_parent # label_item
      })

    hie = {}

    oDiv = self.soup.find("mets:div", {"LABEL": "objects"})

    for div in oDiv.find_all("mets:div"):
      label = div.get("LABEL")
      parent = div.parent
      pid = parent.get("LABEL")
      hie[label] = pid

    self.structs = structs

    self.hie = hie

    if self.debug:
      print("-----\nstructs\n-----")
      pprint(structs)

    if self.debug:
      print("-----\nhie\n-----")
      pprint(hie)

  def getFileSec(self):
    """fileSecの情報の取得
    """

    ## mets:fileGrpの情報の取得

    fileSec = self.soup.find_all("mets:fileGrp")[0]

    files = fileSec.find_all("mets:file")

    file_map = {}

    for file in files:
      file_id = file.get("ID")
      # print(file_id)
      flocat = file.find("mets:FLocat").get("xlink:href")
      # print(flocat)
      file_map[file_id] = flocat

    if self.debug:
      print("-----\nfile_map\n-----")
      pprint(file_map)

    self.file_map = file_map

  def createOmeka(self):
    """Omekaへの登録用のフォーマットに変換
    """
    ## 

    file_map = self.file_map
    structs = self.structs
    items = self.items
    tmp_dir_path = self.tmp_dir_path

    hie = self.hie

    ###

    # params = []
    # rows = []
    # id_exists = []

    # メディア

    medias = []

    for struct in structs:

      # struct: 構造情報を持つ

      item_id = struct["item_id"]

      if "file" in struct:

        file_id = struct["file"]

        # print(file_id, file_id in file_map)

        if file_id not in file_map:
          continue

        path = "objects/" +  file_id.replace("file-", "") + "-" + file_map[file_id].split("/")[-1]

        medias.append({
            "item": item_id, # id,
            "path": path
        })

    item_ids = self.item_ids

    rows = []

    # from copy import deepcopy

    for item_id in item_ids:
      row = items[item_id]
      row["dcterms:identifier"] = item_id
      if item_id in hie:
        row["dcterms:isPartOf ^^resource"] = hie[item_id]
      rows.append(row)

    

    
    df = json_normalize(rows)

    df.to_csv(f'{tmp_dir_path}/metadata.csv', index=False)

    
    df_m = json_normalize(medias)
    df_m

    # print(df_m)

    df_m.to_csv(f'{tmp_dir_path}/media.csv', index=False)

  
  def moveObjects(self):
    """Omekaへの登録用のフォーマットに変換
    """
    ## 
    path = glob.glob(f"{self.tmp_dir_path}/*/objects")[0]
    shutil.move(path, f"{self.tmp_dir_path}")

  @staticmethod
  def convert(dip_zip_file_path, mapping_json_file_path, task_id="bcd"):
    """Omekaへの登録用のフォーマットに変換

    Parameters:
    * `dip_zip_file_path` - path to a dip zip file exported from Archivematica (str or pathlib Path)  
    * `mapping_json_file_path` - path to a mappin file between Archivematica and Omeka (str or pathlib Path)  
    * `task_id` - (Optional) Task ID (str)
    """
    archiveMaticaOmeka = ArchiveMaticaOmeka(dip_zip_file_path, mapping_json_file_path,task_id, debug=False)
    archiveMaticaOmeka.unpackArchive()
    archiveMaticaOmeka.getMetsFilePath()
    
    # print("a", "b")
    
    archiveMaticaOmeka.getDmdSec()
    archiveMaticaOmeka.getStructMap()
    archiveMaticaOmeka.getFileSec()
    archiveMaticaOmeka.createOmeka()
    archiveMaticaOmeka.moveObjects()


    return archiveMaticaOmeka.tmp_dir_path

  @staticmethod
  def upload(metadata_csv_path, media_csv_path, API_URL, KEY_IDENTITY, KEY_CREDENTIAL, is_public=True):
    client = OmekaClient(API_URL, KEY_IDENTITY, KEY_CREDENTIAL)
    client.upload(metadata_csv_path, media_csv_path, is_public=is_public)



In [ ]:
show_doc(ArchiveMaticaOmeka.convert)

---

### ArchiveMaticaOmeka.convert

>      ArchiveMaticaOmeka.convert (dip_zip_file_path, mapping_json_file_path,
>                                  task_id='bcd')

Omekaへの登録用のフォーマットに変換

Parameters:
* `dip_zip_file_path` - path to a dip zip file exported from Archivematica (str or pathlib Path)  
* `mapping_json_file_path` - path to a mappin file between Archivematica and Omeka (str or pathlib Path)  
* `task_id` - (Optional) Task ID (str)

In [ ]:
dip_zip_file_path = "data/sample/ex02-1-TIFF-007c28cc-4671-4d5d-be2a-facace4c7b84.zip"
mapping_json_file_path="data/sample/mapping.json"
output_dir = ArchiveMaticaOmeka.convert(dip_zip_file_path, mapping_json_file_path, task_id="bcd")

In [ ]:
show_doc(ArchiveMaticaOmeka.upload)

---

### ArchiveMaticaOmeka.upload

>      ArchiveMaticaOmeka.upload (metadata_csv_path, media_csv_path, API_URL,
>                                 KEY_IDENTITY, KEY_CREDENTIAL, is_public=True)

In [ ]:
API_URL = os.getenv("API_URL")
KEY_IDENTITY = os.getenv("KEY_IDENTITY")
KEY_CREDENTIAL = os.getenv("KEY_CREDENTIAL")

In [ ]:
metadata_csv_path = f"{output_dir}/metadata.csv"
media_csv_path = f"{output_dir}/media.csv"
ArchiveMaticaOmeka.upload(metadata_csv_path, media_csv_path, API_URL, KEY_IDENTITY, KEY_CREDENTIAL)

アイテムの登録


100%|██████████| 4/4 [00:25<00:00,  6.36s/it]



IDの取得



100%|██████████| 4/4 [00:01<00:00,  2.75it/s]



メディアの登録



100%|██████████| 4/4 [00:34<00:00,  8.66s/it]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

SyntaxError: invalid syntax (<unknown>, line 11)